In [1]:
import os
from shutil import copy
import random
import numpy as np
import pandas as pd
import rasterio
from joblib import Parallel, delayed
from tqdm import tqdm
from time import time, strftime, gmtime

In [2]:
# Copy masked bands to new dir (only: B02, B03, B04)
maindir1 = r'G:\MScThesis\waterQualityMonitoring\Data\GapFilling\preparingS2'
subdir1 = os.path.join(maindir1,'France','Step3_ApplyElevMask')
maindir2 = r'G:\MScThesis\waterQualityMonitoring\Data\GapFilling\massProduction'
subdir2 = os.path.join(maindir2,'France','inputs')

for file1 in os.listdir(subdir1):
    subdir3 = os.path.join(subdir1,file1)
    for file2 in os.listdir(subdir3):
        if 'B02' in file2 or 'B03' in file2 or 'B04' in file2:
            src = os.path.join(subdir3,file2)
            path = os.path.join(subdir2,file1)
            os.makedirs(path, exist_ok=True) # Creates the folder if it does not exist
            dst = os.path.join(path,file2)
            copy(src, dst)  # when using os.path > use copy() # if manually typed a string use copyfile()

# Functions

In [3]:
def find_best_replicate_UV(rowsList_item,colList_item,L1,L2,targetImg,trainingImg,list_boundary_values,groups_KP,list_KP_Values,t,dmax,f):
    i = int(rowsList_item)
    j = int(colList_item)
    lagv = np.array([], dtype=np.int)
    lagh = np.array([], dtype=np.int)
    Nx = []
    ##################################################
    ## Identify Nx, lag vectors, and the data event ##
    ##################################################
    for k1 in L1:
        if i+k1>=0 and i+k1<targetImg.shape[0]:             # Make sure we are within array boundaries
            for k2 in L2:
                if k1 != 0 or k2 != 0:                      # Skip the central point (i,j)
                    if j+k2>=0 and j+k2<targetImg.shape[1]: # Make sure we are within array boundaries
                        Zx = targetImg[int(i+k1),int(j+k2)]
                        if Zx != -99 and Zx < 1e+36:        # need to save only known neihboring values
                            lagv = np.append(lagv, k1)
                            lagh = np.append(lagh, k2)                    
                            Nx = np.append(Nx,Zx)
    ##################################################################################
    ## Search for replicate'y' that has the smallest d(Nx,Ny) in the training image ##
    ##################################################################################

    ###### narrow down search based on which class pixels of Nx are part of ######
    if len(groups_KP) == 1:
        n1 = int(len(groups_KP['list0'])*f)                  # n° of elts to be randomly selected
        random.seed(4)
        subset_list_KP = random.sample(groups_KP['list0'], n1)             # get subset based on specified fraction # Random selection without repetition
    else:
        list_names = []
        list_KP = []
        for value in Nx:
            if value == round(max(list_boundary_values),0):     # round() solved an error attributed to the pc reading 2700.0 as 2699.9999
                list_name_temp = 'list'+str(len(groups_KP)-1)
            else:
                for i in range(len(list_boundary_values)-1): 
                    if (value>=list_boundary_values[i]) and (value<list_boundary_values[i+1]): # identify the closest 2 boundaries
                        list_name_temp = 'list'+str(i)
            list_names.append(list_name_temp) 

        for list_name in list_names:                            # Join all sub-lists that have a pixel in Nx
            list_KP += groups_KP[list_name]

        n1 = int(len(list_KP)*f)                  # n° of elts to be randomly selected
        random.seed(4)
        subset_list_KP = random.sample(list_KP, n1)             # get subset based on specified fraction # Random selection without repetition

    ###### Search for replicate ######
    idx = 999999                                            # initial value until replicate is found
    idy = 999999 
    Zy = 999999 
    ned_temp = 999999
    ned_final = 999999 
    
    for item in subset_list_KP:                             # Search for replicate 'y'
        Ny = []
        for p in range(len(Nx)):
            m1 = item[0] + lagv[p]                          # go up or down
            m2 = item[1] + lagh[p]                          # go left or right
            if (m1>=0) and (m2>=0) and (m1<trainingImg.shape[0]) and (m2<trainingImg.shape[1]): # Make sure we are within array boundaries
                if trainingImg[m1,m2] != -99 and trainingImg[m1,m2] < 1e+36:
                    Ny = np.append(Ny, trainingImg[m1,m2])  # Get a list of neighboring pixel values while excluding Nan and unknown values

        if len(Ny) == len(Nx):                              # Only when Ny and Nx are similar in terms of n° of neighbors, Calculate Normalized Euclidean Distance (NED)
            list_temp = []
            for o in range(len(Ny)):
                list_temp = np.append(list_temp,(Nx[o]-Ny[o])**2)
            ned_temp = np.sqrt(sum(list_temp))/dmax

            if ned_temp < t:
                idx = item[0]
                idy = item[1]
                ned_final = ned_temp                        # Stop searching if the distance threshold is reached
                break
            else:
                if ned_temp < ned_final:
                    idx = item[0]
                    idy = item[1]
                    ned_final = ned_temp                    # In case the distance threshold is not reached, save the smaller number and continue

    if ned_final < 1:                     # If a good match was found (distance between threshold1 and threshold2)                                
        Zy = trainingImg[idx,idy] 
    else:                                  # In case no good match is found OR all known pixels were checked with no replicate to be found (due to not having the same n° of known neighbors: ned_final == 999999)                  
        idx = i
        idy = j       
        ned_final = np.nan
        random.seed()
        Zy = random.sample(list_KP_Values,1)[0]  
     
    return Zy

In [4]:
def find_best_replicate_BV(rowsList_item,colList_item,L1,L2,targetImg,auxImg,list_boundary_values,groups_KP,list_KP_Values1,t,w1,w2,dmax1,dmax2,f):
    i = int(rowsList_item)
    j = int(colList_item)
    lagv1 = np.array([], dtype=np.int)
    lagh1 = np.array([], dtype=np.int)
    Nx1 = []
    lagv2 = np.array([], dtype=np.int)
    lagh2 = np.array([], dtype=np.int)
    Nx2 = []
    ##################################################
    ## Identify Nx, lag vectors, and the data event ##
    ##################################################
    for k1 in L1:
        if i+k1>=0 and i+k1<targetImg.shape[0]: # Make sure we are within array boundaries
            for k2 in L2:
                if k1 != 0 or k2 != 0: # Skip the central point (i,j)
                    if j+k2>=0 and j+k2<targetImg.shape[1]: # Make sure we are within array boundaries
                        Zx1 = targetImg[int(i+k1),int(j+k2)]  # Get pixelValue from training Image                   
                        if Zx1 != -99 and Zx1 < 1e+36: # need to save only known neihboring values
                            lagv1 = np.append(lagv1, k1)
                            lagh1 = np.append(lagh1, k2)                    
                            Nx1 = np.append(Nx1,Zx1)
    for k1 in L1:
        if i+k1>=0 and i+k1<auxImg.shape[0]: # Make sure we are within array boundaries
            for k2 in L2: # Don't skip central point
                if j+k2>=0 and j+k2<auxImg.shape[1]: # Make sure we are within array boundaries
                    Zx2 = auxImg[int(i+k1),int(j+k2)] # Get pixelValue from auxiliary Image               
                    if Zx2 != -99 and Zx2 < 1e+36: # need to save only known neihboring values
                        lagv2 = np.append(lagv2, k1)
                        lagh2 = np.append(lagh2, k2)                    
                        Nx2 = np.append(Nx2,Zx2)
    ##################################################################################
    ## Search for replicate'y' that has the smallest d(Nx,Ny) in the training image ##
    ##################################################################################

    ###### narrow down search based on which class pixels of Nx are part of ######
    if len(groups_KP) == 1:
        n1 = int(len(groups_KP['list0'])*f)                  # n° of elts to be randomly selected
        random.seed(4)
        subset_list_KP = random.sample(groups_KP['list0'], n1)             # get subset based on specified fraction # Random selection without repetition
    else:
        list_names = []
        list_KP = []
        for value in Nx1:
            if value == round(max(list_boundary_values),0):     # round() solved an error attributed to the pc reading 2700.0 as 2699.9999
                list_name_temp = 'list'+str(len(groups_KP)-1)
            else:
                for i in range(len(list_boundary_values)-1): 
                    if (value>=list_boundary_values[i]) and (value<list_boundary_values[i+1]): # identify the closest 2 boundaries
                        list_name_temp = 'list'+str(i)
            list_names.append(list_name_temp) 

        for list_name in list_names:                            # Join all sub-lists that have a pixel in Nx
            list_KP += groups_KP[list_name]

        n1 = int(len(list_KP)*f)                  # n° of elts to be randomly selected
        random.seed(4)
        subset_list_KP = random.sample(list_KP, n1)             # get subset based on specified fraction # Random selection without repetition

    ###### Search for replicate ######
    idx = 999999                                            # initial value until replicate is found
    idy = 999999 
    Zy = 999999 
    ned_temp = 999999
    ned_final = 999999 
    
    for item in subset_list_KP: # Search for replicate 'y'
        Ny1 = []
        Ny2 = []
        for p in range(len(Nx1)):
            m1 = item[0] + lagv1[p] # go up or down
            m2 = item[1] + lagh1[p] # go left or right
            if (m1>=0) and (m2>=0) and (m1<targetImg.shape[0]) and (m2<targetImg.shape[1]): # Make sure we are within array boundaries
                if targetImg[m1,m2] != -99 and targetImg[m1,m2] != 9.96921e+36:
                    Ny1 = np.append(Ny1, targetImg[m1,m2]) # Get a list of neighboring pixel values while excluding Nan and unknown values
        for p in range(len(Nx2)):
            m1 = item[0] + lagv2[p] # go up or down
            m2 = item[1] + lagh2[p] # go left or right
            if (m1>=0) and (m2>=0) and (m1<auxImg.shape[0]) and (m2<auxImg.shape[1]): # Make sure we are within array boundaries
                if auxImg[m1,m2] != -99 and auxImg[m1,m2] != 9.96921e+36:
                    Ny2 = np.append(Ny2, auxImg[m1,m2]) # Get a list of neighboring pixel values while excluding Nan and unknown values

        if (len(Ny1) == len(Nx1)) and (len(Ny2) == len(Nx2)): # Calculate NED
            list_temp1 = []
            for o in range(len(Ny1)):
                list_temp1 = np.append(list_temp1,(Nx1[o]-Ny1[o])**2)
            list_temp2 = []
            for o in range(len(Ny2)):
                list_temp2 = np.append(list_temp2,(Nx2[o]-Ny2[o])**2)

            ned_temp = w1*(np.sqrt(sum(list_temp1))/dmax1)+w2*(np.sqrt(sum(list_temp2))/dmax2) # sum of weights should be 1 # weights reflect importance

            if ned_temp < t:
                idx = item[0]
                idy = item[1]
                ned_final = ned_temp # Stop searching if the distance threshold is reached
                break
            else:
                if ned_temp < ned_final:
                    idx = item[0]
                    idy = item[1]
                    ned_final = ned_temp # In case the distance threshold is not reached, save the smaller number and continue
    
    if ned_final < 1:                     # If a good match was found (distance between threshold1 and threshold2)                                
        Zy = targetImg[idx,idy] 
    else:                                  # In case no good match is found OR all known pixels were checked with no replicate to be found (due to not having the same n° of known neighbors: ned_final == 999999)                  
        idx = i
        idy = j       
        ned_final = np.nan
        random.seed()
        Zy = random.sample(list_KP_Values1,1)[0]  
    
    return Zy

In [5]:
def get_coordKP(item, array):    
    if array[item[0],item[1]] != -99:
        return item
def generate_list(offset): 
    L = list(range(offset+1))
    L1 = [-item for item in L if item != 0]
    L += L1 
    return L
def Convert_To_Offsets(x):
    return {'8': [1,1],'14':[1,2],'20':[1,3],'24':[2,2],'34':[2,3],'38':[3,3],'44':[2,4],'62':[3,4],'80':[4,4]}.get(x, [2,2])   # 24 is default if x not found
def get_known_values(list_item, array):     # Calculate dmax from all known pixels in training image
    pixelValue = array[list_item[0],list_item[1]]
    return pixelValue


In [6]:
def selectUP_minNx(coord_item,array,L1,L2,minNx):# need to adopt the same L1/L2 so that calculating avg(Nx) won't give NaN
    i = coord_item[0]
    j = coord_item[1]
    Nx = []
    for k1 in L1:
        if i+k1>=0 and i+k1<array.shape[0]:      # Make sure we are within array boundaries
            for k2 in L2:
                if k1 != 0 or k2 != 0:           # Skip the central point (i,j)
                    if j+k2>=0 and j+k2<array.shape[1]: # Make sure we are within array boundaries
                        if array[int(i+k1),int(j+k2)] != -99 and array[int(i+k1),int(j+k2)] < 1e+36:
                            Nx = np.append(Nx, array[int(i+k1),int(j+k2)])
    if len(Nx)>=minNx:
        return i,j

# Mass Production

In [7]:
########## Parameters ##########
n1 = str(8)              # Define a n° of neighbors # Available options: 8/14/20/24/34/38/44/62/80 # Default: 24
t = 0.1               # Lower distance Threshold
f = 0.2                 # how much of the training image will be searched (float number between 0 and 1)
n2 = 100                 # N° defined sub-groups
w1 = 0.5              # sum of w1 & w2 need to be equal to 1
w2 = 0.5
minNx = 3               # Min n° of neighbors for a selected UP before starting the algorithm
start = 72
finish = 84 # 0; 12; 24; 36; 48; 60; 72; 84; 90

In [8]:
# Read S2 images (6 by 6)
maindir1 = r'G:\MScThesis\waterQualityMonitoring\Data\GapFilling\massProduction'
subdir1 = os.path.join(maindir1,'France','inputs')
outputdir = os.path.join(maindir1,'France','outputs')
Bands = ['B02','B03','B04']

for Band in Bands:
    for i in range(0,len(os.listdir(subdir1)[start:finish]),6):
        file_dates = np.array([]).astype('str')
        for j in range(6):
            file0 = os.listdir(subdir1)[start:finish][i+j]    
            file_dates = np.append(file_dates, file0)
        # Set Target & Before/After Images
        for k in tqdm(range(1,5)):
            Target_date = file_dates[k]
            Before_date = file_dates[k-1]
            After_date = file_dates[k+1]
            # Read as arrays
            for file1 in os.listdir(os.path.join(subdir1,Target_date)): 
                if Band in file1:
                    img = rasterio.open(os.path.join(subdir1,Target_date,file1))
                    targetImg = rasterio.open(os.path.join(subdir1,Target_date,file1)).read(1)
                    target_fileName = file1
            for file2 in os.listdir(os.path.join(subdir1,Before_date)):
                if Band in file2:
                    my_array1 = rasterio.open(os.path.join(subdir1,Before_date,file2)).read(1)
            for file3 in os.listdir(os.path.join(subdir1,After_date)):
                if Band in file3:
                    my_array2 = rasterio.open(os.path.join(subdir1,After_date,file3)).read(1)        

            ########## Get coordUP & Get coordKP & Identify training image to be used ##########
            coordMP = np.argwhere(targetImg < 1e+36).tolist()                # MP: Mixed Pixels
            coord_KP = Parallel(n_jobs=-1)(delayed(get_coordKP)(coordMP[k],targetImg) for k in range(len(coordMP)))
            coordKP = [x for x in coord_KP if x != None]                  # KP: Known Pixels
            coordUP = np.argwhere(targetImg == -99).tolist()                    # UP: Unknown Pixels

            offsetV = Convert_To_Offsets(n1)[0]          # offset rows
            offsetH = Convert_To_Offsets(n1)[1]          # offset cols
            L1 = generate_list(offsetV)                 # Define all possible movements (forward or backward)
            L2 = generate_list(offsetH)                 # Define all possible movements (forward or backward)

            if len(coordKP) >= len(coordUP): # Apply UV version
                trainingImg = targetImg.copy() 

                ########## calculate dmax ##########
                list_KP_Values = Parallel(n_jobs=-1)(delayed(get_known_values)(coordKP[x1],trainingImg) for x1 in range(len(coordKP)))
                Max_value = max(list_KP_Values)
                Min_value = min(list_KP_Values)
                dmax = Max_value - Min_value

                ########## Divide coordKP into n2 ##########
                # Determine the values representing the boundaries of each of the n2
                Range_known_values = dmax/n2
                boundary_value = Min_value
                list_boundary_values = [Min_value]
                for k in range(n2):
                    boundary_value+=Range_known_values               # divided into equal groups
                    list_boundary_values = np.append(list_boundary_values, boundary_value)
                for k in range(n2):
                    boundary_value+=Range_known_values               
                    list_boundary_values = np.append(list_boundary_values, boundary_value)

                list_KP_Values = Parallel(n_jobs=-1)(delayed(get_known_values)(coordKP[x1],targetImg) for x1 in range(len(coordKP)))
                if min(list_boundary_values)>Min_value:
                    list_boundary_values[0]=Min_value
                if max(list_boundary_values)<Max_value:
                    list_boundary_values[-1]=Max_value

                # initialize a dictionnary of empty lists to store coord of known pixels corresponding to each N_group
                list_names=['list'+str(i) for i in range(len(list_boundary_values)-1)]
                groups_KP = {}
                for list_name in list_names:
                    groups_KP[list_name] = []

                # Fill the lists with corresponding coord of known pixel values
                for k in range(len(list_KP_Values)):                 # Determine to which group the selected value corresponds to 
                    if list_KP_Values[k] == max(list_boundary_values):
                        groups_KP['list'+str(n2-1)].append(coordKP[k])
                    else:
                        for i in range(len(list_boundary_values)-1): # identify the closest 2 boundaries
                            if (list_KP_Values[k]>=list_boundary_values[i]) and (list_KP_Values[k]<list_boundary_values[i+1]):
                                groups_KP['list'+str(i)].append(coordKP[k])

                NumberUP = len(np.argwhere(targetImg == -99).tolist())
                while (NumberUP != 0):
                    # Get indice of unknown pixels with at least 3 immediate neighbors (v=1, h=1)
                    coordUP = np.argwhere(targetImg == -99).tolist() # Get indices of unknown pixels    
                    res = Parallel(n_jobs=-1)(delayed(selectUP_minNx)(coordUP[k],targetImg,L1,L2,minNx) for k in range(len(coordUP)))
                    selectedUP = [x for x in res if x != None]
                    if len(selectedUP) != 0:                                     # In case there are UP with the indicated minNx condition
                        random.seed(5)
                        reorderedUP = random.sample(selectedUP, len(selectedUP)) # need to reorder, so that the predicted value of unknown pixel 'x2' is not hugely affected by the previously 'x1'
                        rowsList, colList = zip(*reorderedUP)                    # unpack results
                    else:                                                        # In case there are no UP with  with the indicated minNx condition
                        if len(coordUP) > 5:
                            random.seed(5)        
                            selectedUP = random.sample(coordUP, 5)
                            rowsList, colList = zip(*selectedUP) # unpack results  
                        else:
                            random.seed(5)        
                            selectedUP = random.sample(coordUP, 1)
                            rowsList, colList = zip(*selectedUP) # unpack results 

                    ########## Search for best replicate using all laptop cores (in the selected sub-group) ##########
                    results = Parallel(backend='loky',n_jobs=-1,verbose=0)(delayed(find_best_replicate_UV)(rowsList[iteration],colList[iteration],L1,L2,targetImg,trainingImg,list_boundary_values,groups_KP,list_KP_Values,t,dmax,f) for iteration in range(len(rowsList)))

                    ########## Update Arrays with new simulated values ##########
                    for item in range(len(rowsList)):
                        targetImg[int(rowsList[item]),int(colList[item])] = results[item]

                    trainingImg = targetImg.copy()
                    NumberUP = len(np.argwhere(targetImg == -99).tolist())

            else:  # Apply BV version
                ########## Identify auxiliary image to be used ##########
                coordMP_arr1 = np.argwhere(my_array1 < 1e+36).tolist()  # Get indices of known and NaN pixels
                coord_KP_arr1 = Parallel(n_jobs=-1)(delayed(get_coordKP)(coordMP_arr1[k],my_array1) for k in range(len(coordMP_arr1)))
                coordKP_arr1 = [x for x in coord_KP_arr1 if x != None]

                coordMP_arr2 = np.argwhere(my_array2 < 1e+36).tolist()  # Get indices of known and NaN pixels
                coord_KP_arr2 = Parallel(n_jobs=-1)(delayed(get_coordKP)(coordMP_arr2[k],my_array2) for k in range(len(coordMP_arr2)))
                coordKP_arr2 = [x for x in coord_KP_arr2 if x != None]

                if len(coordKP_arr1) > len(coordKP_arr2): # change between arr1 / arr2 / arr3
                    auxImg = my_array1.copy()
                    used_aux_Img = 'before'
                else:
                    auxImg = my_array2.copy()
                    used_aux_Img = 'after'

                ########## calculate dmax ##########
                list_KP_Values1 = Parallel(n_jobs=-1)(delayed(get_known_values)(coordKP[x1],targetImg) for x1 in range(len(coordKP)))
                if len(list_KP_Values1)>0:                
                    Max_value1 = max(list_KP_Values1)
                    Min_value1 = min(list_KP_Values1)
                    dmax1 = Max_value1 - Min_value1
                    list_KP_Values2 = Parallel(n_jobs=-1)(delayed(get_known_values)(coordKP[x1],auxImg) for x1 in range(len(coordKP))) # The central point will be taken from the targetImg >> need to use the same coorKP of targetImg on auxImg
                    Max_value2 = max(list_KP_Values2)
                    Min_value2 = min(list_KP_Values2)
                    dmax2 = Max_value2 - Min_value2

                    ########## Divide coordKP into n2 ##########
                    # Determine the values representing the boundaries of each of the n2
                    Range_known_values = dmax1/n2
                    boundary_value = Min_value1
                    list_boundary_values = [Min_value1]
                    for k in range(n2):
                        boundary_value+=Range_known_values               # divided into equal groups
                        list_boundary_values = np.append(list_boundary_values, boundary_value)

                    # initialize a dictionnary of empty lists to store coord of known pixels corresponding to each N_group
                    list_names=['list'+str(i) for i in range(len(list_boundary_values)-1)]
                    groups_KP = {}
                    for list_name in list_names:
                        groups_KP[list_name] = []

                    # Fill the lists with corresponding coord of known pixel values
                    for k in range(len(list_KP_Values1)):                 # Determine to which group the selected value corresponds to 
                        if list_KP_Values1[k] == max(list_boundary_values):
                            groups_KP['list'+str(n2-1)].append(coordKP[k])
                        else:
                            for i in range(len(list_boundary_values)-1): # identify the closest 2 boundaries
                                if (list_KP_Values1[k]>=list_boundary_values[i]) and (list_KP_Values1[k]<list_boundary_values[i+1]):
                                    groups_KP['list'+str(i)].append(coordKP[k])

                    NumberUP = len(np.argwhere(targetImg == -99).tolist())
                    while (NumberUP != 0):            
                        # Get indice of unknown pixels with at least 3 immediate neighbors (v=1, h=1)
                        coordUP = np.argwhere(targetImg == -99).tolist() # Get indices of unknown pixels    
                        res = Parallel(n_jobs=-1)(delayed(selectUP_minNx)(coordUP[k],targetImg,L1,L2,minNx) for k in range(len(coordUP)))
                        selectedUP = [x for x in res if x != None]
                        if len(selectedUP) != 0:                                     # In case there are UP with the indicated minNx condition
                            random.seed(5)
                            reorderedUP = random.sample(selectedUP, len(selectedUP)) # need to reorder, so that the predicted value of unknown pixel 'x2' is not hugely affected by the previously 'x1'
                            rowsList, colList = zip(*reorderedUP)                    # unpack results
                        else:                                                        # In case there are no UP with  with the indicated minNx condition
                            if len(coordUP) > 5:
                                random.seed(5)        
                                selectedUP = random.sample(coordUP, 5)
                                rowsList, colList = zip(*selectedUP) # unpack results  
                            else:
                                random.seed(5)        
                                selectedUP = random.sample(coordUP, 1)
                                rowsList, colList = zip(*selectedUP) # unpack results

                        ########## Search for best replicate using all laptop cores (in the selected sub-group) ##########
                        results = Parallel(backend='loky',n_jobs=-1,verbose=0)(delayed(find_best_replicate_BV)(rowsList[iteration],colList[iteration],L1,L2,targetImg,auxImg,list_boundary_values,groups_KP,list_KP_Values1,t,w1,w2,dmax1,dmax2,f) for iteration in range(len(rowsList)))

                        ########## Update Arrays with new simulated values ##########
                        for item in range(len(rowsList)):
                            targetImg[int(rowsList[item]),int(colList[item])] = results[item]
                        NumberUP = len(np.argwhere(targetImg == -99).tolist())
                else:
                    # Get avg arr 1 and arr2
                    for i in range(targetImg.shape[0]):
                        for j in range(targetImg.shape[1]):
                            c1 = my_array1[i,j]>0 and my_array1[i,j]<1e+36
                            c2 = my_array2[i,j]>0 and my_array2[i,j]<1e+36
                            if c1 == True and c2 == True:
                                targetImg[i,j] = (my_array1[i,j]+my_array2[i,j])/2
                            else:
                                if c1 == True and c2 == False:
                                    targetImg[i,j] = my_array1[i,j]
                                else:
                                    if c1 == False and c2 == True:
                                        targetImg[i,j] = my_array2[i,j]
                                    else:
                                        if c1 == False and c2 == False:
                                            targetImg[i,j] = 9.96921e+36    

            ########## Save gap filled images ##########
            path = os.path.join(outputdir,Target_date)
            os.makedirs(path, exist_ok=True) # Creates the folder if it does not exist
            outputdir1 = os.path.join(path, 'GF_'+target_fileName)
            with rasterio.open(outputdir1,'w',driver='Gtiff', width=img.width, height=img.height, 
                                count=1,crs=img.crs,transform=img.transform, dtype='float32', nodata=9.96921e+36) as newImg:
                newImg.write(targetImg,1)
                newImg.close()

100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [5:48:57<00:00, 5234.37s/it]
